In [1]:
from typing import List
from copy import deepcopy

import collections
import matplotlib.pyplot as plt; plt.rcParams.update({"font.family": "serif"})

import pyscf
import pyscf.cc
import pyscf.mcscf

# To get molecular geometries.
import openfermion as of
from openfermion import MolecularData
from openfermionpyscf import run_pyscf

import qiskit
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.primitives import BitArray
from qiskit_aer import AerSimulator  # For MPS Simulator.

import ffsim

# To run on hardware.
import qiskit_ibm_runtime
from qiskit_ibm_runtime import SamplerV2 as Sampler

from functools import partial

import numpy as np

from qiskit_addon_sqd.fermion import SCIResult, diagonalize_fermionic_hamiltonian, solve_sci_batch

from adaptvqe.pools import DVG_CEO, FullPauliPool
from adaptvqe.hamiltonians import HubbardHamiltonian
from adaptvqe.algorithms.adapt_vqe import LinAlgAdapt, TensorNetAdapt

In [2]:
ibm_computer: str = "ibm_fez"

service = qiskit_ibm_runtime.QiskitRuntimeService(channel="local")
computer = service.backend()
sampler = Sampler(computer)

/Users/benjamindalfavero/.venv/adapt/lib/python3.13/site-packages/qiskit_ibm_runtime/fake_provider/backends/nighthawk/fake_nighthawk.py:76: UserWarning: Properties of fake_nighthawk are not intended to represent typical nighthawk error values.
  warnings.warn(


In [3]:
l = 2
sys_size = l * l
t = 1.0
u = 4.0
hamiltonian = HubbardHamiltonian(l, l, t, u, False, True)

exact_energy = hamiltonian.ground_energy
print(f"Ground state energy = {exact_energy}")

Ground state energy = -6.102748483462062


In [4]:
def neel_circuit(nq):
    circuit = QuantumCircuit(nq)
    for i in range(nq):
        if i % 2 == 0:
            circuit.x(i)
        else:
            circuit.id(i)
    return circuit

In [5]:
pool = FullPauliPool(n=sys_size)

max_mpo_bond = 200
adapt_mps_bond = 10
my_adapt = TensorNetAdapt(
    pool=pool,
    custom_hamiltonian=hamiltonian,
    max_adapt_iter=1,
    recycle_hessian=True,
    tetris=True,
    verbose=True,
    threshold=0.1,
    max_mpo_bond=max_mpo_bond,
    max_mps_bond=adapt_mps_bond
)

my_adapt.initialize()
nq = my_adapt.n

circuits = []
adapt_energies = []
for i in range(6):
    print(f"On iteration {i}.")
    my_adapt.run_iteration()
    data = my_adapt.data
    ansatz_circuit = pool.get_circuit(my_adapt.indices, my_adapt.coefficients)
    print("coefficients:", my_adapt.coefficients)
    print("indices:", my_adapt.indices)
    # Prepare the HF reference state, then add the Ansatz circuit.
    q = QuantumRegister(nq)
    circuit = QuantumCircuit(q)
    # circuit.append(ffsim.qiskit.PrepareHartreeFockJW(num_orbitals, nelec), q)
    circuit = circuit.compose(neel_circuit(nq))
    circuit = circuit.compose(ansatz_circuit)
    circuit.measure_all()
    circuits.append(circuit)
    adapt_energies.append(my_adapt.energy)


tensor-net-adapt prepared with the following settings:
> Pool: full_pauli_pool
> Custom Hamiltonian: HH_2_2_1.0_4.0
> Orbital Optimization: False
> Selection method: gradient
> Convergence criterion: total_g_norm
> Recycling Hessian: True
> Tetris: True (progressive optimization: False)
> Convergence threshold (gradient norm):  0.1
> Maximum number of iterations:  1
> candidates per iteration:  1

Initial energy: -4.0000000000000036
On iteration 0.

*** ADAPT-VQE Iteration 1 ***

Creating list of up to 255 operators ordered by gradient magnitude...

Non-Zero Gradients (tolerance E-8):
Operator 38: -2.0000000000000036
Operator 39: -2.0000000000000036
Operator 54: 2.0000000000000036
Operator 55: 2.0000000000000036
Operator 98: 2.0000000000000036
Operator 99: 2.0000000000000036
Operator 114: -2.0000000000000036
Operator 115: -2.0000000000000036
Operator 137: 2.0000000000000013
Operator 141: -2.0000000000000013
Operator 152: -2.0000000000000013
Operator 156: 2.0000000000000013
Operator 20

/Users/benjamindalfavero/Documents/phd/wellcome/ceo-adapt-vqe/adaptvqe/algorithms/adapt_vqe.py:2492: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  opt_result = minimize_bfgs(
/Users/benjamindalfavero/Documents/phd/wellcome/ceo-adapt-vqe/adaptvqe/algorithms/adapt_vqe.py:2507: UserWarning: Optimizer did not succeed. Message: Desired error not necessarily achieved due to precision loss.
  warn(f"Optimizer did not succeed. Message: {opt_result.message}")


         Current function value: -4.236068
         Iterations: 6
         Function evaluations: 35
         Gradient evaluations: 24

Current energy: -4.236067977499787
(change of -0.2360679774997836)
Current ansatz: [38]
coefficients: [np.float64(0.23182377058830256)]
indices: [38]
On iteration 1.

*** ADAPT-VQE Iteration 2 ***

Creating list of up to 255 operators ordered by gradient magnitude...

Non-Zero Gradients (tolerance E-8):
Operator 4: 0.8944270696722953
Operator 21: 0.8944270696722953
Operator 38: -3.0331905002956017e-07
Operator 39: -3.0331905002956017e-07
Operator 54: 3.0331905002956017e-07
Operator 55: 3.0331905002956017e-07
Operator 64: -0.8944270696722953
Operator 81: -0.8944270696722953
Operator 98: 3.0331905002956017e-07
Operator 99: 3.0331905002956017e-07
Operator 114: -3.0331905002956017e-07
Operator 115: -3.0331905002956017e-07
Operator 137: 1.7888544426636388
Operator 141: -2.0000000000000018
Operator 152: -1.7888544426636388
Operator 156: 2.0000000000000018
Ope

/Users/benjamindalfavero/Documents/phd/wellcome/ceo-adapt-vqe/adaptvqe/algorithms/adapt_vqe.py:2492: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  opt_result = minimize_bfgs(
/Users/benjamindalfavero/Documents/phd/wellcome/ceo-adapt-vqe/adaptvqe/algorithms/adapt_vqe.py:2507: UserWarning: Optimizer did not succeed. Message: Desired error not necessarily achieved due to precision loss.
  warn(f"Optimizer did not succeed. Message: {opt_result.message}")
/Users/benjamindalfavero/Documents/phd/wellcome/ceo-adapt-vqe/adaptvqe/algorithms/adapt_vqe.py:2492: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  opt_result = minimize_bfgs(
/Users/benjamindalfavero/Documents/phd/wellcome/ceo-adapt-vqe/adaptvqe/algorithms/adapt_vqe.py:2507: UserWarning: Optimizer did not succeed. Message: Desired error not necessarily achieved due to precision loss.
  warn(f"Optimizer did not succeed. Message: {opt_result.message}")


         Current function value: -4.828427
         Iterations: 12
         Function evaluations: 44
         Gradient evaluations: 34

Current energy: -4.828427124732341
(change of -0.32842712473232627)
Current ansatz: [38, 141, 1]
coefficients: [np.float64(1.6844781307100838e-06), np.float64(0.3926992094468564), np.float64(0.7853983713914718)]
indices: [38, 141, 1]
On iteration 3.

*** ADAPT-VQE Iteration 4 ***

Creating list of up to 255 operators ordered by gradient magnitude...

Non-Zero Gradients (tolerance E-8):
Operator 1: 5.882960874004084e-07
Operator 16: -5.882960874004084e-07
Operator 38: 1.4453050078411245e-06
Operator 39: 8.319760933517273e-07
Operator 54: -8.319760933517273e-07
Operator 55: -1.4453050078411245e-06
Operator 69: 5.882960874004084e-07
Operator 84: -5.882960874004084e-07
Operator 98: -1.4453050078411245e-06
Operator 99: -8.319760933517273e-07
Operator 114: 8.319760933517273e-07
Operator 115: 1.4453050078411245e-06
Operator 137: -1.4453052529228572e-06
Operat

ValueError: not enough values to unpack (expected 2, got 1)